# 2.7 学習と検証の実施

- 本ファイルでは、SSDの学習と検証の実施を行います。手元のマシンで動作を確認後、AWSのGPUマシンで計算します。
- p2.xlargeで約6時間かかります。


# 学習目標

1.	SSDの学習を実装できるようになる

# 事前準備

- AWS EC2 のGPUインスタンスを使用します
- フォルダ「utils」のssd_model.pyをします

In [1]:
# パッケージのimport
import os.path as osp
import random
import time

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [2]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

使用デバイス： cuda:0


# DatasetとDataLoaderを作成する

In [4]:
from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn


# ファイルパスのリストを取得
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath)

# Datasetを作成
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  # (BGR)の色の平均値
input_size = 300  # 画像のinputサイズを300×300にする

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))


# DataLoaderを作成する
batch_size = 32

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


# ネットワークモデルの作成する

In [5]:
from utils.ssd_model import SSD

# SSD300の設定
ssd_cfg = {
    'num_classes': 21,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# SSDネットワークモデル
net = SSD(phase="train", cfg=ssd_cfg)

# SSDの初期の重みを設定
# 他のノートでは行ってない追加部分なことに注意
# ssdのvgg部分に重みをロードする
vgg_weights = torch.load('./weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)

# ssdのその他のネットワークの重みはHeの初期値で初期化
# Heの初期値は活性化関数がReLUの場合に使用する初期化手法

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:  # バイアス項がある場合
            nn.init.constant_(m.bias, 0.0)


# Heの初期値を適用
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

# GPUが使えるかを確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", device)

print('ネットワーク設定完了：学習済みの重みをロードしました')


使用デバイス： cuda:0
ネットワーク設定完了：学習済みの重みをロードしました


# 損失関数と最適化手法を定義する

In [6]:
from utils.ssd_model import MultiBoxLoss

# 損失関数の設定
criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)

# 最適化手法の設定
optimizer = optim.SGD(net.parameters(), lr=1e-3,
                      momentum=0.9, weight_decay=5e-4)


# 学習・検証を実施する

In [7]:
# モデルを学習させる関数を作成

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # イテレーションカウンタをセット
    iteration = 1
    epoch_train_loss = 0.0  # epochの損失和
    epoch_val_loss = 0.0  # epochの損失和
    logs = []

    # epochのループ
    for epoch in range(num_epochs+1):

        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
                print('（train）')
            else:
                if((epoch+1) % 10 == 0):
                    net.eval()   # モデルを検証モードに
                    print('-------------')
                    print('（val）')
                else:
                    # 検証は10回に1回だけ行う
                    continue

            # データローダーからminibatchずつ取り出すループ
            for images, targets in dataloaders_dict[phase]:

                # GPUが使えるならGPUにデータを送る
                images = images.to(device)
                targets = [ann.to(device)
                           for ann in targets]  # リストの各要素のテンソルをGPUへ

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    # 順伝搬（forward）計算
                    outputs = net(images)

                    # 損失の計算
                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()  # 勾配の計算

                        # 勾配が大きくなりすぎると計算が不安定になるので、clipで最大でも勾配2.0に留める
                        nn.utils.clip_grad_value_(
                            net.parameters(), clip_value=2.0)

                        optimizer.step()  # パラメータ更新

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 検証時
                    else:
                        epoch_val_loss += loss.item()

        # epochのphaseごとのlossと正解率
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # ログを保存
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

        epoch_train_loss = 0.0  # epochの損失和
        epoch_val_loss = 0.0  # epochの損失和

        # ネットワークを保存する
        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), 'weights/ssd300_' +
                       str(epoch+1) + '.pth')


In [8]:
# 学習・検証を実行する
num_epochs= 50  
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用デバイス： cuda:0
-------------
Epoch 1/50
-------------
（train）
イテレーション 10 || Loss: 16.7851 || 10iter: 50.8701 sec.
イテレーション 20 || Loss: 12.0824 || 10iter: 23.8051 sec.
イテレーション 30 || Loss: 10.2823 || 10iter: 24.8721 sec.
イテレーション 40 || Loss: 10.7739 || 10iter: 24.7633 sec.
イテレーション 50 || Loss: 8.7581 || 10iter: 24.3612 sec.
イテレーション 60 || Loss: 8.0017 || 10iter: 24.0850 sec.
イテレーション 70 || Loss: 8.7532 || 10iter: 24.3068 sec.
イテレーション 80 || Loss: 7.9840 || 10iter: 24.7281 sec.
イテレーション 90 || Loss: 8.0331 || 10iter: 24.2647 sec.
イテレーション 100 || Loss: 7.3940 || 10iter: 24.5302 sec.
イテレーション 110 || Loss: 7.7241 || 10iter: 25.4273 sec.
イテレーション 120 || Loss: 7.7591 || 10iter: 24.8209 sec.
イテレーション 130 || Loss: 7.5527 || 10iter: 24.4328 sec.
イテレーション 140 || Loss: 7.5964 || 10iter: 23.9767 sec.
イテレーション 150 || Loss: 7.6153 || 10iter: 24.8465 sec.
イテレーション 160 || Loss: 7.8187 || 10iter: 24.9340 sec.
イテレーション 170 || Loss: 7.4967 || 10iter: 24.4809 sec.
-------------
epoch 1 || Epoch_TRAIN_Loss:1646.5532 ||Epoch

イテレーション 1380 || Loss: 5.8069 || 10iter: 24.7251 sec.
イテレーション 1390 || Loss: 5.5085 || 10iter: 24.4228 sec.
イテレーション 1400 || Loss: 5.5607 || 10iter: 24.3720 sec.
イテレーション 1410 || Loss: 5.6586 || 10iter: 24.4984 sec.
イテレーション 1420 || Loss: 5.0669 || 10iter: 25.0218 sec.
イテレーション 1430 || Loss: 4.8530 || 10iter: 24.8773 sec.
-------------
epoch 8 || Epoch_TRAIN_Loss:964.9154 ||Epoch_VAL_Loss:0.0000
timer:  465.8547 sec.
-------------
Epoch 9/50
-------------
（train）
イテレーション 1440 || Loss: 5.5850 || 10iter: 19.4693 sec.
イテレーション 1450 || Loss: 5.0659 || 10iter: 24.8459 sec.
イテレーション 1460 || Loss: 5.3466 || 10iter: 24.4323 sec.
イテレーション 1470 || Loss: 5.3750 || 10iter: 24.8586 sec.
イテレーション 1480 || Loss: 5.4853 || 10iter: 24.7612 sec.
イテレーション 1490 || Loss: 4.7816 || 10iter: 24.8094 sec.
イテレーション 1500 || Loss: 5.2384 || 10iter: 24.8651 sec.
イテレーション 1510 || Loss: 4.5730 || 10iter: 24.3556 sec.
イテレーション 1520 || Loss: 5.1399 || 10iter: 24.3046 sec.
イテレーション 1530 || Loss: 4.8939 || 10iter: 24.7350 sec.
イテレーション 

イテレーション 2710 || Loss: 4.0296 || 10iter: 24.9843 sec.
イテレーション 2720 || Loss: 4.6026 || 10iter: 24.9698 sec.
イテレーション 2730 || Loss: 4.6333 || 10iter: 24.5967 sec.
イテレーション 2740 || Loss: 4.8766 || 10iter: 25.4349 sec.
イテレーション 2750 || Loss: 4.7765 || 10iter: 24.8020 sec.
イテレーション 2760 || Loss: 4.5850 || 10iter: 24.7714 sec.
イテレーション 2770 || Loss: 4.3459 || 10iter: 24.7688 sec.
イテレーション 2780 || Loss: 4.5986 || 10iter: 24.7625 sec.
イテレーション 2790 || Loss: 4.7289 || 10iter: 24.9447 sec.
イテレーション 2800 || Loss: 4.3632 || 10iter: 25.0357 sec.
イテレーション 2810 || Loss: 4.7897 || 10iter: 25.0112 sec.
イテレーション 2820 || Loss: 4.5338 || 10iter: 25.3394 sec.
イテレーション 2830 || Loss: 5.1947 || 10iter: 24.3259 sec.
イテレーション 2840 || Loss: 4.8836 || 10iter: 24.7442 sec.
イテレーション 2850 || Loss: 3.8930 || 10iter: 24.6303 sec.
イテレーション 2860 || Loss: 4.7016 || 10iter: 24.3938 sec.
-------------
epoch 16 || Epoch_TRAIN_Loss:811.3210 ||Epoch_VAL_Loss:0.0000
timer:  467.0542 sec.
-------------
Epoch 17/50
-------------
（train）
イテレーショ

イテレーション 4060 || Loss: 4.1133 || 10iter: 24.8684 sec.
イテレーション 4070 || Loss: 3.9143 || 10iter: 24.5010 sec.
イテレーション 4080 || Loss: 4.0226 || 10iter: 25.1854 sec.
イテレーション 4090 || Loss: 4.6322 || 10iter: 24.6537 sec.
イテレーション 4100 || Loss: 4.2907 || 10iter: 24.6690 sec.
イテレーション 4110 || Loss: 4.1977 || 10iter: 25.2640 sec.
-------------
epoch 23 || Epoch_TRAIN_Loss:741.6633 ||Epoch_VAL_Loss:0.0000
timer:  464.2886 sec.
-------------
Epoch 24/50
-------------
（train）
イテレーション 4120 || Loss: 3.9509 || 10iter: 6.7164 sec.
イテレーション 4130 || Loss: 4.3013 || 10iter: 25.0571 sec.
イテレーション 4140 || Loss: 4.0523 || 10iter: 24.8978 sec.
イテレーション 4150 || Loss: 3.6229 || 10iter: 25.4055 sec.
イテレーション 4160 || Loss: 3.8499 || 10iter: 24.2695 sec.
イテレーション 4170 || Loss: 4.2206 || 10iter: 24.4929 sec.
イテレーション 4180 || Loss: 4.0013 || 10iter: 24.3307 sec.
イテレーション 4190 || Loss: 4.1358 || 10iter: 24.1477 sec.
イテレーション 4200 || Loss: 4.0536 || 10iter: 24.8488 sec.
イテレーション 4210 || Loss: 4.3615 || 10iter: 24.7114 sec.
イテレーション

イテレーション 5390 || Loss: 4.0460 || 10iter: 24.3877 sec.
イテレーション 5400 || Loss: 3.7143 || 10iter: 24.1303 sec.
イテレーション 5410 || Loss: 3.5034 || 10iter: 24.5186 sec.
イテレーション 5420 || Loss: 3.9109 || 10iter: 24.2815 sec.
イテレーション 5430 || Loss: 3.9490 || 10iter: 25.1970 sec.
イテレーション 5440 || Loss: 4.3588 || 10iter: 24.3267 sec.
イテレーション 5450 || Loss: 3.4736 || 10iter: 25.7239 sec.
イテレーション 5460 || Loss: 4.1169 || 10iter: 25.7325 sec.
イテレーション 5470 || Loss: 4.5913 || 10iter: 24.2134 sec.
イテレーション 5480 || Loss: 3.6747 || 10iter: 24.7130 sec.
イテレーション 5490 || Loss: 3.6483 || 10iter: 24.6064 sec.
イテレーション 5500 || Loss: 3.2627 || 10iter: 25.2369 sec.
イテレーション 5510 || Loss: 3.1410 || 10iter: 24.3648 sec.
イテレーション 5520 || Loss: 4.0504 || 10iter: 24.4509 sec.
イテレーション 5530 || Loss: 3.4650 || 10iter: 24.3802 sec.
イテレーション 5540 || Loss: 3.6521 || 10iter: 24.6183 sec.
-------------
epoch 31 || Epoch_TRAIN_Loss:684.6100 ||Epoch_VAL_Loss:0.0000
timer:  465.1294 sec.
-------------
Epoch 32/50
-------------
（train）
イテレーショ

イテレーション 6750 || Loss: 3.4073 || 10iter: 24.7877 sec.
イテレーション 6760 || Loss: 3.8127 || 10iter: 25.0141 sec.
イテレーション 6770 || Loss: 3.4649 || 10iter: 24.5148 sec.
イテレーション 6780 || Loss: 3.5333 || 10iter: 25.1514 sec.
イテレーション 6790 || Loss: 3.8567 || 10iter: 24.9513 sec.
イテレーション 6800 || Loss: 3.3921 || 10iter: 24.7852 sec.
-------------
epoch 38 || Epoch_TRAIN_Loss:651.5282 ||Epoch_VAL_Loss:0.0000
timer:  465.9067 sec.
-------------
Epoch 39/50
-------------
（train）
イテレーション 6810 || Loss: 3.9910 || 10iter: 19.7338 sec.
イテレーション 6820 || Loss: 3.3931 || 10iter: 24.8883 sec.
イテレーション 6830 || Loss: 3.4093 || 10iter: 26.4614 sec.
イテレーション 6840 || Loss: 3.5849 || 10iter: 25.2506 sec.
イテレーション 6850 || Loss: 3.7817 || 10iter: 25.2425 sec.
イテレーション 6860 || Loss: 3.5971 || 10iter: 24.7020 sec.
イテレーション 6870 || Loss: 3.5761 || 10iter: 24.6620 sec.
イテレーション 6880 || Loss: 3.7340 || 10iter: 24.6686 sec.
イテレーション 6890 || Loss: 3.7733 || 10iter: 24.7227 sec.
イテレーション 6900 || Loss: 3.5220 || 10iter: 25.0531 sec.
イテレーショ

イテレーション 8080 || Loss: 3.0240 || 10iter: 24.5301 sec.
イテレーション 8090 || Loss: 3.2341 || 10iter: 24.4193 sec.
イテレーション 8100 || Loss: 3.4679 || 10iter: 24.7412 sec.
イテレーション 8110 || Loss: 3.7881 || 10iter: 24.5195 sec.
イテレーション 8120 || Loss: 3.4604 || 10iter: 24.8207 sec.
イテレーション 8130 || Loss: 2.6310 || 10iter: 24.2920 sec.
イテレーション 8140 || Loss: 3.6508 || 10iter: 25.9052 sec.
イテレーション 8150 || Loss: 3.5706 || 10iter: 24.2916 sec.
イテレーション 8160 || Loss: 3.4823 || 10iter: 25.5785 sec.
イテレーション 8170 || Loss: 3.6990 || 10iter: 24.7270 sec.
イテレーション 8180 || Loss: 3.4395 || 10iter: 24.8425 sec.
イテレーション 8190 || Loss: 3.3660 || 10iter: 24.8085 sec.
イテレーション 8200 || Loss: 3.3595 || 10iter: 24.5095 sec.
イテレーション 8210 || Loss: 3.3622 || 10iter: 24.6952 sec.
イテレーション 8220 || Loss: 3.2182 || 10iter: 24.8860 sec.
イテレーション 8230 || Loss: 2.8953 || 10iter: 24.3362 sec.
-------------
epoch 46 || Epoch_TRAIN_Loss:612.4380 ||Epoch_VAL_Loss:0.0000
timer:  466.3960 sec.
-------------
Epoch 47/50
-------------
（train）
イテレーショ

以上